# 環境設定

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

--2019-12-27 08:52:08--  https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6bnwn8u2hz19s59/init_env.sh [following]
--2019-12-27 08:52:08--  https://www.dropbox.com/s/raw/6bnwn8u2hz19s59/init_env.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5c581e04eeaf19370f11bf6250.dl.dropboxusercontent.com/cd/0/inline/AvCfuXKa2-ivo5wR54U4Cnq9KfTEipobY5OJgEmyxC7yn-f0vVTCxcX-LW36NGAA5kcqz_DpygREFyyakLrlamJcQNzMo9tmOAefK1JRO6i7ww/file# [following]
--2019-12-27 08:52:09--  https://uc5c581e04eeaf19370f11bf6250.dl.dropboxusercontent.com/cd/0/inline/AvCfuXKa2-ivo5wR54U4Cnq9KfTEipobY5OJgEmyxC7yn-f0vVTCxcX-LW36NGAA5kcqz_DpygREFyyakLrlamJcQNzMo9tmOAefK1JRO6i7ww/file
Resolving uc5c581e04ee

In [0]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [0]:
from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext()

# --------預處理部分--------


# 讀取資料

In [0]:
!wget -O pm25.csv "https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0"

--2019-12-27 08:52:18--  https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv [following]
--2019-12-27 08:52:18--  https://www.dropbox.com/s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc57ac1a82af0ab942729b3e228f.dl.dropboxusercontent.com/cd/0/inline/AvCJfjkoyQFkLr6QGBXOWkneF8z2kjv8BDERNvBYx1GnfK_Bz0hGwvw3WjR-EDyr0C79LKUfiEOs99JmT582eG2WsgOxUPYBPoOwN2-3qygolg/file# [following]
--2019-12-27 08:52:18--  https://uc57ac1a82af0ab942729b3e228f.dl.dropboxusercontent.com/cd/0/inline/AvCJfjkoyQFkLr6QGBXOWkneF8z2kjv8BDERNvBYx1GnfK_Bz0hGwvw3WjR-EDyr0C79LKUfiEOs99JmT582eG2WsgOxUPYBPoOwN2-3qygolg/file
Resol

In [0]:
weather_data = sc.textFile("pm25.csv")

In [0]:
weather_data_rdd = weather_data.map(lambda line : line.split(","))

In [0]:
pm25schema = weather_data_rdd.first()
print (pm25schema)

['日期', '測站', '測項', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


In [0]:
import math
def remove_row_with_noise (x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True 

In [0]:
clean_weather_data = weather_data_rdd\
                    .filter(lambda x: x!=pm25schema)\
                    .filter(remove_row_with_noise)

In [0]:
#print(set(clean_weather_data.map(lambda x : x[1]).collect()))

In [0]:
list1 = list(set(clean_weather_data.map(lambda x : x[1]).collect()))

In [0]:
#str1 =''
#for i in range(75):
#  str2 = "x[1] == '" + list1[i] + "' or "
#  str1 += str2
#print(str1[:-3])

In [0]:
#len(set(clean_weather_data.map(lambda x : x[1]).collect()))

In [0]:
dalipm25 = clean_weather_data

In [0]:
#dalipm25 = clean_weather_data.filter(lambda x :x[1] == '士林' or x[1] == '埔里' or x[1] == '小港' or x[1] == '觀音' or x[1] == '花蓮' or x[1] == '松山' or x[1] == '新竹' or x[1] == '陽明' or x[1] == '新莊' or x[1] == '安南' or x[1] == '忠明' or x[1] == '屏東' or x[1] == '平鎮' or x[1] == '頭份' or x[1] == '金門' or x[1] == '前金' or x[1] == '中山' or x[1] == '善化' or x[1] == '崙背' or x[1] == '淡水' or x[1] == '林園' or x[1] == '冬山' or x[1] == '馬公' or x[1] == '鳳山' or x[1] == '沙鹿' or x[1] == '潮州' or x[1] == '菜寮' or x[1] == '新營' or x[1] == '新店' or x[1] == '仁武' or x[1] == '臺西' or x[1] == '彰化' or x[1] == '三義' or x[1] == '汐止' or x[1] == '苗栗' or x[1] == '關山' or x[1] == '大里' or x[1] == '湖口' or x[1] == '南投' or x[1] == '馬祖' or x[1] == '宜蘭' or x[1] == '板橋' or x[1] == '美濃' or x[1] == '橋頭' or x[1] == '斗六' or x[1] == '復興' or x[1] == '大寮' or x[1] == '楠梓' or x[1] == '龍潭' or x[1] == '大同' or x[1] == '豐原' or x[1] == '朴子' or x[1] == '嘉義' or x[1] == '麥寮' or x[1] == '桃園' or x[1] == '新港' or x[1] == '左營' or x[1] == '基隆' or x[1] == '土城' or x[1] == '古亭' or x[1] == '中壢' or x[1] == '永和' or x[1] == '萬里' or x[1] == '西屯' or x[1] == '前鎮' or x[1] == '二林' or x[1] == '林口' or x[1] == '臺南' or x[1] == '線西' or x[1] == '臺東' or x[1] == '大園' or x[1] == '三重' or x[1] == '竹東' or x[1] == '恆春' or x[1] == '萬華' )

In [0]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
dalipm25row = dalipm25.map(lambda p:
        Row(
        date = p[0],
        location = p[1],
        measure = p[2],
        hr_01 = float(p[3]),hr_02 = float(p[4]),hr_03 = float(p[5]),hr_04 = float(p[6]),hr_05 = float(p[7]),
        hr_06 = float(p[8]),hr_07 = float(p[9]),hr_08 = float(p[10]),hr_09 = float(p[11]),hr_10 = float(p[12]),
        hr_11 = float(p[13]),hr_12 = float(p[14]),hr_13 = float(p[15]),hr_14 = float(p[16]),hr_15 = float(p[17]),
        hr_16 = float(p[18]),hr_17 = float(p[19]),hr_18 = float(p[20]),hr_19 = float(p[21]),hr_20 = float(p[22]),
        hr_21 = float(p[23]),hr_22 = float(p[24]),hr_23 = float(p[25]),hr_24 = float(p[26]),
    )
)


#dalipm25row.take(10)

In [0]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [0]:
df = spark.createDataFrame(dalipm25row)

In [0]:
#df.show()

In [0]:
schemea = ["date", "location", "measure", 
           "hr_01", "hr_02", "hr_03", "hr_04",
           "hr_05", "hr_06", "hr_07", "hr_08",
           "hr_09", "hr_10", "hr_11", "hr_12",
           "hr_13", "hr_14", "hr_15", "hr_16",
           "hr_17", "hr_18", "hr_19", "hr_20",
           "hr_21", "hr_22", "hr_23", "hr_24",           
          ]

In [0]:
df = dalipm25.toDF(schemea)

In [0]:
#df.show()

In [0]:
for i in df.columns[3:]:
  df = df.withColumn(i, df[i].cast("float"))

In [0]:
#df.show()

In [0]:
df_pm10 = df.select(\
                    df.date.alias("datepm10"),\
                    df.hr_09.alias("hr_09_pm10"),\
                    df.hr_10.alias("hr_10_pm10"),\
                    df.hr_11.alias("hr_11_pm10"),\
                    df.hr_12.alias("hr_12_pm10"),\
                    "measure")\
                    .filter(df.measure=="PM10")


In [0]:
#df_pm10.show()

In [0]:
df_pm25 = df.select(\
                    df.date.alias("datepm25"),\
                    df.hr_09.alias("hr_09_pm25"),\
                    df.hr_10.alias("hr_10_pm25"),\
                    df.hr_11.alias("hr_11_pm25"),\
                    df.hr_12.alias("hr_12_pm25"),\
                    "measure")\
                    .filter(df.measure=="PM2.5") 


In [0]:
#df_pm25.show()

In [0]:
traing_data = df_pm25\
    .join(df_pm10, df_pm25.datepm25==df_pm10.datepm10)\
    .drop("measure")\
    .drop("datepm10")

In [0]:
#traing_data.show()

In [0]:
#traing_data.corr("hr_09_NO2", "hr_09_")

In [0]:
from pyspark.sql import functions as F
formulated_traning_data = traing_data\
            .select("*", F.when(traing_data.hr_12_pm25 > 50, 1).otherwise(0))\
            .drop("hr_12_pm10")\
            .drop("hr_12_pm25")\
            .drop("datepm25")

In [0]:
#formulated_traning_data.show()

In [0]:
from pyspark.sql import functions as F
formulated_traning_data = traing_data\
            .select("*", F.when(traing_data.hr_12_pm25 > 50, 1).otherwise(0))\
            .withColumnRenamed("CASE WHEN (hr_12_pm25 > 50) THEN 1 ELSE 0 END", "pm25_condiction")\
            .drop("hr_12_pm10")\
            .drop("hr_12_pm25")\
            .drop("datepm25")

In [0]:
#formulated_traning_data.show()

# 預處理Model

In [0]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils


# Decision Tree Model

In [0]:
from pyspark.mllib.regression import LabeledPoint

LabelPoints = formulated_traning_data.rdd\
            .map(lambda r: LabeledPoint(r.pm25_condiction, \
                                        [r.hr_09_pm25, r.hr_10_pm25, r.hr_11_pm25, \
                                         r.hr_09_pm10, r.hr_10_pm10, r.hr_11_pm10 ]))


In [0]:
(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0]) 

In [0]:
DTModel = DecisionTree.trainClassifier(trainData,
        numClasses=2,
        categoricalFeaturesInfo={},
        impurity="entropy",
        maxDepth=20,
        maxBins=5)

In [0]:
# Save model
DTModel.save(sc, "content/drive/My Drive/colab/model/DTModel")

# Random Forest Model

In [0]:
RFModel = RandomForest.trainClassifier(trainData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=4, featureSubsetStrategy="auto",
                                     impurity='entropy', maxDepth=20, maxBins=5)

In [0]:
# Save model
RFModel.save(sc, "content/drive/My Drive/colab/model/RFModel")

# Regression

In [0]:
regression_training = \
             traing_data.drop("hr_12_pm10")\
            .drop("datepm25")

In [0]:
from pyspark.mllib.regression import LabeledPoint

LabelPoints = regression_training.rdd\
            .map(lambda r: LabeledPoint(r.hr_12_pm25, \
                                        [r.hr_09_pm25, r.hr_10_pm25, r.hr_11_pm25, \
                                         r.hr_09_pm10, r.hr_10_pm10, r.hr_11_pm10]))

In [0]:
(trainData,validationData,testData) = LabelPoints.randomSplit([10,0,0])

In [0]:
REModel = DecisionTree.trainRegressor(trainData, categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=5, maxBins=32)

In [0]:
# Save model
REModel.save(sc, "content/drive/My Drive/colab/model/REModel")

# 測資處理

In [0]:
!wget "https://www.dropbox.com/s/3tejnx4i3v2fv0v/pm%2025%20test%20samples.zip?dl=0"

--2019-12-27 08:15:13--  https://www.dropbox.com/s/3tejnx4i3v2fv0v/pm%2025%20test%20samples.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3tejnx4i3v2fv0v/pm%2025%20test%20samples.zip [following]
--2019-12-27 08:15:14--  https://www.dropbox.com/s/raw/3tejnx4i3v2fv0v/pm%2025%20test%20samples.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce0043527e727decccdbf4757dc.dl.dropboxusercontent.com/cd/0/inline/AvDYlc8tkKWB9wADmiArEJcil0YrOZ8U3b21PZ-GX2Q0crD4ZZrI8gBubMx-z1mLXWePkvVRscTDQkzHMzaKOsA3D7akVsILaGAtGq3SN1zHIIN2bp-k9boIr8ixj2JD_UQ/file# [following]
--2019-12-27 08:15:14--  https://uce0043527e727decccdbf4757dc.dl.dropboxusercontent.com/cd/0/inline/AvDYlc8tkKWB9wADmiArEJcil0YrOZ8U3b21PZ-GX2Q0crD4ZZrI8gBubMx-z

In [0]:
!unzip 'pm 25 test samples.zip?dl=0'

Archive:  pm 25 test samples.zip?dl=0
replace 2015_04_01_loc5_PM2.5_23.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2015_04_01_loc5_PM2.5_23.csv  
replace __MACOSX/._2015_04_01_loc5_PM2.5_23.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._2015_04_01_loc5_PM2.5_23.csv  
replace 2015_11_01_loc1_PM2.5_11.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2015_11_01_loc1_PM2.5_11.csv  
replace __MACOSX/._2015_11_01_loc1_PM2.5_11.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._2015_11_01_loc1_PM2.5_11.csv  
replace 2015_05_01_loc4_PM2.5_03.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2015_05_01_loc4_PM2.5_03.csv  
replace __MACOSX/._2015_05_01_loc4_PM2.5_03.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._2015_05_01_loc4_PM2.5_03.csv  
replace 2015_05_05_loc3_PM2.5_05.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2015_05_05_loc3_PM2.5_05.csv  
replace __MACOSX/._2015_05_05_loc3_PM2.5_0

In [0]:
!ls

 2015_04_01_loc5_PM2.5_23.csv   content      'pm 25 test samples.zip?dl=0'
 2015_05_01_loc4_PM2.5_03.csv   drive	     'pm 25 test samples.zip?dl=0.1'
 2015_05_05_loc3_PM2.5_05.csv   init_env.sh   sample_data
 2015_07_12_loc2_PM2.5_11.csv   __MACOSX      spark-2.2.0-bin-hadoop2.7.tgz
 2015_11_01_loc1_PM2.5_11.csv   pm25.csv      spark-warehouse


In [0]:
!ls /content/drive/My\ Drive/colab/pm25_testdata/

001.csv  002.csv  003.csv  004.csv  005.csv


In [0]:
def vote(choose):
  if sum(choose)>=2:
    ans = 1
  else:
    ans = 0
  return ans

In [0]:
from pyspark.mllib.regression import LabeledPoint
import time

ans = "["
total_test = 5

for i in range(total_test):
  num = i+1
  print('Round: ',num)
  if num < 10:
    file_num = '/content/drive/My Drive/colab/pm25_testdata/00' + str(num) + '.csv'
  elif num >= 10 and num < 100:
    file_num = '/content/drive/My Drive/colab/pm25_testdata/0' + str(num) + '.csv'
  else:
    file_num = '/content/drive/My Drive/colab/pm25_testdata/' + str(num) + '.csv'
  print(file_num)
  # get start time
  start = time.time()

  # read file
  test_data = sc.textFile(file_num)
  test_data_rdd = test_data.map(lambda line : line.split(","))
  test_data_rdd = test_data_rdd.filter(lambda x: x[2] == 'PM2.5' or x[2] == 'PM10')

  # get list
  tt_data = test_data_rdd.collect()

  ttt_data = tt_data[-1] #pm25 this day
  ttt_pm10_data = tt_data[-2] #pm10 this day
  ttt_data_yes = tt_data[-3]
  ttt_pm10_yes = tt_data[-4]
  print(ttt_data)
  print(ttt_pm10_data)
  print(ttt_data_yes)
  print(ttt_pm10_yes)
  predict = 1
  for i in range(-1,-25,-1):
    if ttt_data[i] != '':
      predict = (i + 26) % 24 
      break
  print(predict)

  if predict == 1:
    ii = 24
    only1data = ttt_data[ii:]
    onlypm10data = ttt_pm10_data[ii:]
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6)
    
  if predict == 2:
    only1data = ttt_data[3]
    onlypm10data = ttt_pm10_data[3]
    temp = ttt_data_yes[-2:]
    temp.extend(only1data)
    only1data = temp
    temp = ttt_pm10_yes[-2:]
    temp.extend(onlypm10data)
    onlypm10data = temp
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6) 

  if predict == 3:
    only1data = ttt_data[3:5]
    onlypm10data = ttt_pm10_data[3:5]
    temp = ttt_data_yes[-1]
    temp.extend(only1data)
    only1data = temp
    temp = ttt_pm10_yes[-1]
    temp.extend(onlypm10data)
    onlypm10data = temp
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6) 

    
  if predict > 3 :
    ii = predict + 2
    only1data = ttt_data[ii-3:ii]
    onlypm10data = ttt_pm10_data[ii-3:ii]
    print(only1data)
    print(onlypm10data)
    only1data.extend(onlypm10data)
    only1data_rdd = sc.parallelize(only1data)
    only1data_rdd = only1data_rdd.map(lambda x: float(x))
    only1data_rdd.take(6)

  
  only1_data = LabeledPoint(1,only1data)
  only1_data

  # Decision Tree Model
  Load_DTModel = DecisionTreeModel.load(sc, "content/drive/My Drive/colab/model/DTModel")
  prediction_DT = Load_DTModel.predict(only1_data.features)
  print('Prediction_DT: ', int(prediction_DT))

  # Random Forest Model
  Load_RFModel = RandomForestModel.load(sc, "content/drive/My Drive/colab/model/RFModel")
  prediction_RF = Load_RFModel.predict(only1_data.features)
  print('Prediction_RF: ', int(prediction_RF))

  # Regression Model
  Load_REModel = DecisionTreeModel.load(sc, "content/drive/My Drive/colab/model/REModel")
  prediction_data = Load_REModel.predict(only1_data.features)
  print('Regression Model: ', prediction_data)

  # determine while > 50
  if(prediction_data>50):
    prediction_RE=1
  else:
    prediction_RE=0
  print('Prediction_RE: ', prediction_RE)

  choose=[]
  choose.append(int(prediction_DT))
  choose.append(int(prediction_RF))
  choose.append(prediction_RE)
  print('Choose result: ', choose)
  # vote result
  vote_result = vote(choose)
  print('Vote result: ', vote(choose))
  
  # update ans
  ans += str(vote_result)
  if num != total_test:
    ans += ","
  else:
    ans += "]"

  # " "
  """
  ans += "\""
  ans += str(vote_result)
  if num != total_test:
    ans += "\","
  else:
    ans += "\"]"
  """

  # get end time
  end = time.time()
  print('Period: ' + str(round(end-start,2)) + 's')
  print("\n\n")

print("\n##############################################################################\n")
print("Answer: ",ans)

Round:  1
/content/drive/My Drive/colab/pm25_testdata/001.csv
['2015/04/01', ' ', 'PM2.5', '34', '35', '36', '26', '25', '31', '26', '28', '28', '26', '23', '24', '37', '34', '31', '20', '27', '20', '19', '14', '16', '16', '18', '24']
['2015/04/01', ' ', 'PM10', '88', '99', '99', '86', '86', '88', '81', '64', '61', '60', '71', '82', '81', '82', '69', '72', '67', '64', '61', '60', '60', '62', '61', '60']
['2015/03/31', ' ', 'PM2.5', '20', '15', '18', '16', '20', '21', '27', '27', '28', '30', '29', '26', '27', '23', '20', '20', '19', '23', '12', '20', '13', '20', '16', '20']
['2015/03/31', ' ', 'PM10', '58', '50', '43', '48', '45', '53', '54', '66', '68', '67', '63', '61', '60', '60', '58', '69', '73', '72', '58', '56', '59', '63', '63', '67']
1
['16', '18', '24']
['62', '61', '60']
Prediction_DT:  0
Prediction_RF:  0
Regression Model:  23.406565319396496
Prediction_RE:  0
Choose result:  [0, 0, 0]
Vote result:  0
Period: 4.06s



Round:  2
/content/drive/My Drive/colab/pm25_testdata/002